In [1]:
# autoreload modules
%load_ext autoreload
%autoreload 2

In [ ]:
import bempp.api
import numpy as np

## Test Assembly

In [2]:
import MyHM.assembly as asb

In [3]:
# grid = bempp.api.shapes.sphere(h=0.2)
# grid = bempp.api.shapes.ellipsoid(r1=0.5, r2=1, r3=0.3, origin=(0, 0, 0), h=0.1)
# grid = bempp.api.shapes.ellipsoid(3, 1, 1, h=h)
# grid = bempp.api.shapes.cube(h=1)
# grid = bempp.api.shapes.cube(h=0.5)
grid = bempp.api.import_grid('grids/ribcage4-h-4.msh')
# grid = bempp.api.import_grid('grids/ribcage3-h-1.msh')

# grid.plot()
print(grid.vertices.shape)
print(grid.elements.shape)

# bempp.api.DEFAULT_DEVICE_INTERFACE = 'opencl'
bempp.api.DEFAULT_DEVICE_INTERFACE = 'numba'


(3, 1751)
(3, 3486)


In [4]:
space = bempp.api.function_space(grid, "P", 1)
k = 7
dlp = bempp.api.operators.boundary.helmholtz.double_layer(space, space, space, k)
hyp = bempp.api.operators.boundary.helmholtz.hypersingular(space, space, space, k)
boundary_operator = dlp
%time matrix_wf = boundary_operator.weak_form()
A = np.array(matrix_wf.A)

CPU times: user 1min, sys: 228 ms, total: 1min
Wall time: 18.4 s


In [8]:
# Inputs:
parameters = bempp.api.GLOBAL_PARAMETERS
device_interface = "numba"
# device_interface = "opencl"

# Block rows and columns:
cols = list(range(4,10))
rows = list(range(0,4))

# cols = list(range(grid.vertices.shape[1]))
# rows = list(range(grid.vertices.shape[1]))

result = np.zeros((len(rows), len(cols)), dtype=np.complex128)

# Assemble dense:
%time asb.partial_dense_assembler(boundary_operator.descriptor, boundary_operator.domain, boundary_operator.dual_to_range, parameters, result, cols=cols, rows=rows)
%time singular_sm = asb.singular_assembler_sparse(device_interface, boundary_operator.descriptor, boundary_operator.domain, boundary_operator.dual_to_range, parameters)  

CPU times: user 18.7 ms, sys: 116 µs, total: 18.8 ms
Wall time: 8.36 ms
CPU times: user 2.97 s, sys: 9.9 ms, total: 2.98 s
Wall time: 505 ms


In [9]:
%%time
meshgrid = np.meshgrid(rows, cols, indexing="ij")
result += singular_sm[meshgrid[0], meshgrid[1]]

CPU times: user 934 µs, sys: 6 µs, total: 940 µs
Wall time: 727 µs


In [10]:
# Check results:
print(np.linalg.norm(result - A[meshgrid[0], meshgrid[1]]))
print(np.linalg.norm(result - A[meshgrid[0], meshgrid[1]]) / np.linalg.norm(A[meshgrid[0], meshgrid[1]]))

6.646539573125444e-17
9.308623842080668e-17


## Test Trees

In [ ]:
# Hacer min_block_size

In [24]:
import MyHM.structures as stt

In [25]:
# grid = bempp.api.shapes.sphere(h=0.2)
# grid = bempp.api.shapes.ellipsoid(r1=0.5, r2=1, r3=0.3, origin=(0, 0, 0), h=0.1)
# grid = bempp.api.shapes.ellipsoid(3, 1, 1, h=h)
# grid = bempp.api.shapes.cube(h=1)
# grid = bempp.api.shapes.cube(h=0.5)
grid = bempp.api.import_grid('grids/ribcage4-h-4.msh')
# grid = bempp.api.import_grid('grids/ribcage3-h-1.msh')

# grid.plot()
print(grid.vertices.shape)
print(grid.elements.shape)


(3, 1751)
(3, 3486)


In [26]:
bbox = grid.bounding_box
vertices = grid.vertices
space = bempp.api.function_space(grid, "P", 1)
if not space.requires_dof_transformation:
    dof_indices = list(range(vertices.shape[1]))
else:
    # TODO:
    raise NotImplementedError
    # space.dof_transformation.indices

octree = stt.Octree(vertices.T, dof_indices, bbox, max_depth=4)
%time octree.generate_tree()

tree_3d = stt.Tree3D(octree)
%time tree_3d.generate_adm_tree()

CPU times: user 18.4 ms, sys: 0 ns, total: 18.4 ms
Wall time: 17.8 ms
CPU times: user 1.82 s, sys: 90.4 ms, total: 1.91 s
Wall time: 1.89 s


In [27]:
tree_3d.stats

{'number_of_nodes': 44963,
 'number_of_leaves': 41652,
 'number_of_not_adm_leaves': 17630,
 'number_of_adm_leaves': 24022}

In [ ]:
# Agregar la implementación del assembly en el árbol!
# Verificar que no hayas nodos hojas admisibles con partes singulares!
# Una vez hecho lo anterior, hacer gráfico de normas de diferencias entre matriz original y comprimida a medida que
# disminuimos el valor de epsilon.